In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('notebook')

%load_ext autoreload
%autoreload 2

In [ ]:
from crawler import async_cache_pages, urls_list

In [ ]:
# edX does not work as some content seems to be dynamically downloaded
folders = ["coursera", "general"]

positives, negatives = urls_list(folders)
async_cache_pages(positives + negatives)

In [ ]:
feature_functions = {
        #"url": url,
        #"label": label,
        "a_count": lambda soup: len(soup.find_all("a")),
        "iframe_count": lambda soup: len(soup.find_all("iframe")),
        "h1_count": lambda soup: len(soup.find_all("h1")),
        "h2_count": lambda soup: len(soup.find_all("h2")),
        "h3_count": lambda soup: len(soup.find_all("h3")),
        "video_count": lambda soup: len(soup.find_all("video")),
        "button_count": lambda soup: len(soup.find_all("button"))
        
    }

In [ ]:
from bs4 import BeautifulSoup
from crawler import get_cached, is_cached

def construct_df_features(urls, labels, features):
    row_list = []
    features_func = [feature_functions[feat] for feat in features]
    columns = ["url", "label"] + features

    for url, label in zip(urls, labels):
        if is_cached(url):
            soup = BeautifulSoup(get_cached(url), "lxml")

        row = [url, label] + [func(soup) for func in features_func]

        row_list.append(row)

    df = pd.DataFrame(row_list, columns=columns)
    df = df.set_index("url")
    
    return df

In [ ]:
urls = [positives[0], negatives[0]]
labels = [True, False]
features = ["button_count", "a_count"]

construct_df_features(urls, labels, features)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

In [ ]:
all_features = sorted(list(feature_functions.keys()))

In [ ]:
def cross_val(df, selected_features, estimator=RandomForestClassifier(), cv=3):
    X = df[selected_features]
    y = df["label"]
    
    return cross_val_score(rf_estimator, X, y, cv=cv)

In [ ]:
urls_pos, urls_neg = urls_list(["coursera", "general"])
urls = urls_pos + urls_neg
labels = [True] * len(urls_pos) + [False] * len(urls_neg)

results_means = []
results_stds = []

df = construct_df_features(urls, labels, all_features)

# iterate over all features and train on this feature alone
for feature in all_features:
    # Cross val with the selected feature alone
    scores = cross_val(df, [feature])
    
    results_means.append(np.mean(scores))
    results_stds.append(np.std(scores))

In [ ]:
# Same thing, but with same ratio of positive and negative examples

import random
random.seed(0)

urls_pos, urls_neg = urls_list(["coursera", "general"])

# Same number of negative and positive examples
urls = urls_pos + random.sample(urls_neg, len(urls_pos))
labels = [True] * len(urls_pos) + [False] * len(urls_pos)

results_unbiased_means = []
results_unbiased_stds = []

df_unbiased = construct_df_features(urls, labels, all_features)

# iterate over all features and train on one feature alone
for feature in all_features:
    # Cross val with the selected feature alone
    scores = cross_val(df_unbiased, [feature])
    
    results_unbiased_means.append(np.mean(scores))
    results_unbiased_stds.append(np.std(scores))

In [ ]:
import os

index = np.arange(len(results_means))
fig, ax = plt.subplots(figsize=(18, 8))
bar_width = 0.35
axis_font_size = 18


plt.bar(index, results_means, bar_width, yerr=results_stds, color='tab:red', label="Biased estimator")
plt.xlabel("Feature", fontsize=axis_font_size)
plt.ylabel("Cross validation score", fontsize=axis_font_size)
plt.xticks(index, all_features)

plt.bar(index + bar_width, results_unbiased_means, bar_width, yerr=results_unbiased_stds, label="Unbiased estimator")
plt.xlabel("Feature", fontsize=axis_font_size)
plt.ylabel("Cross validation score", fontsize=axis_font_size)
plt.xticks(index + bar_width / 2, all_features, fontsize=14)

plt.axhline(y=0.5, color='tab:brown', alpha=0.7, linewidth=1)
plt.legend(fontsize=15)

plt.savefig(os.path.join("figures", "feature_importance_biased_estimator"))